In [1]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os
    import sys

    USE_CASES = "/var/SP/data/home/bgmerin1/src/Repositorios/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))

    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))
set_paths()

Added '/var/SP/data/home/bgmerin1/src/Repositorios/use-cases' to path
Added '/var/SP/data/home/bgmerin1/src/Repositorios' to path


In [3]:
# coding=utf-8

import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import sys
import time
from pyspark.sql.functions import (udf,
                                    col,
                                    decode,
                                    when,
                                    lit,
                                    lower,
                                    concat,
                                    translate,
                                    count,
                                    sum as sql_sum,
                                    max as sql_max,
                                    min as sql_min,
                                    avg as sql_avg,
                                    greatest,
                                    least,
                                    isnull,
                                    isnan,
                                    struct, 
                                    substring,
                                    size,
                                    length,
                                    year,
                                    month,
                                    dayofmonth,
                                    unix_timestamp,
                                    date_format,
                                    from_unixtime,
                                    datediff,
                                    to_date, 
                                    desc,
                                    asc,
                                    countDistinct,
                                    row_number,
                                    skewness,
                                    kurtosis,
                                    concat_ws,
                                    upper)

from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType, FloatType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from datetime import datetime
from itertools import chain
import numpy as np
from functools import reduce
from churn.models.fbb_churn_amdocs.utils_general import *
from pykhaos.utils.date_functions import convert_to_date
from churn.models.fbb_churn_amdocs.utils_model import *
from churn.models.fbb_churn_amdocs.utils_fbb_churn import *
from churn.models.fbb_churn_amdocs.metadata_fbb_churn import *
from churn.models.fbb_churn_amdocs.feature_selection_utils import *
import subprocess


In [4]:
# Para que no haga falta recargar el notebook si hacemos un cambio en alguna de las funciones que estamos importando
%load_ext autoreload
%autoreload 2

In [5]:
# set BDP parameters
def setting_bdp(min_n_executors = 3, max_n_executors = 15, n_cores = 8, executor_memory = "12g", driver_memory="8g",
                   app_name = "Python app", driver_overhead="1g", executor_overhead='3g'):

    MAX_N_EXECUTORS = max_n_executors
    MIN_N_EXECUTORS = min_n_executors
    N_CORES_EXECUTOR = n_cores
    EXECUTOR_IDLE_MAX_TIME = 120
    EXECUTOR_MEMORY = executor_memory
    DRIVER_MEMORY = driver_memory
    N_CORES_DRIVER = 1
    MEMORY_OVERHEAD = N_CORES_EXECUTOR * 2048
    QUEUE = "root.BDPtenants.es.medium"
    BDA_CORE_VERSION = "1.0.0"

    SPARK_COMMON_OPTS = os.environ.get('SPARK_COMMON_OPTS', '')
    SPARK_COMMON_OPTS += " --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
    SPARK_COMMON_OPTS += " --conf spark.shuffle.manager=tungsten-sort"
    SPARK_COMMON_OPTS += "  --queue %s" % QUEUE

    # Dynamic allocation configuration
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.shuffle.service.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.executor.cores=%s" % (N_CORES_EXECUTOR)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
    # SPARK_COMMON_OPTS += " --conf spark.ui.port=58235"
    SPARK_COMMON_OPTS += " --conf spark.port.maxRetries=100"
    SPARK_COMMON_OPTS += " --conf spark.app.name='%s'" % (app_name)
    SPARK_COMMON_OPTS += " --conf spark.submit.deployMode=client"
    SPARK_COMMON_OPTS += " --conf spark.ui.showConsoleProgress=true"
    SPARK_COMMON_OPTS += " --conf spark.sql.broadcastTimeout=1200"
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.memoryOverhead={}".format(executor_overhead)
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.driverOverhead={}".format(driver_overhead)

    BDA_ENV = os.environ.get('BDA_USER_HOME', '')

    # Attach bda-core-ra codebase
    SPARK_COMMON_OPTS+=" --files {}/scripts/properties/red_agent/nodes.properties,{}/scripts/properties/red_agent/nodes-de.properties,{}/scripts/properties/red_agent/nodes-es.properties,{}/scripts/properties/red_agent/nodes-ie.properties,{}/scripts/properties/red_agent/nodes-it.properties,{}/scripts/properties/red_agent/nodes-pt.properties,{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

    os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
    os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS
   

In [6]:
def get_spark_session(app_name="default name", log_level='INFO', min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    HOME_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src")
    if HOME_SRC not in sys.path:
        sys.path.append(HOME_SRC)


    setting_bdp(app_name=app_name, min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores, executor_memory = executor_memory, driver_memory=driver_memory)
    from common.src.main.python.utils.hdfs_generic import run_sc
    sc, spark, sql_context = run_sc(log_level=log_level)


    return sc, spark, sql_context

In [7]:
def initialize(app_name, min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    import time
    start_time = time.time()

    print("_initialize spark")
    #import pykhaos.utils.pyspark_configuration as pyspark_config
    sc, spark, sql_context = get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores,
                             executor_memory = executor_memory, driver_memory=driver_memory)
    print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                         sc.defaultParallelism))
    return spark

In [8]:
spark = initialize("PBMA Incrementales v2",executor_memory = "32g",min_n_executors = 6,max_n_executors = 15)


_initialize spark
Ended spark session: 54.3324978352 secs | default parallelism=2


In [ ]:
## Averias

def averias(spark,starting_day,closing_day):

    ga_tickets = spark.read.table('raw_es.callcentrecalls_ticketsow')
    ga_tickets_detalle = spark.read.table('raw_es.callcentrecalls_ticketdetailow')
    ga_franquicia = spark.read.table('raw_es.callcentrecalls_ticketfranchiseow')
    ga_close_case = spark.read.table('raw_es.callcentrecalls_ticketclosecaseow')
    clientes = spark.read.table('raw_es.customerprofilecar_customerow').select('OBJID', 'NIF_CLIENTE')
    ga_tipo_tickets = spark.read.parquet('/data/raw/vf_es/cvm/GATYPETICKETS/1.0/parquet')
 
    from pyspark.sql.functions import lpad
 
    from pyspark.sql.functions import year, month, dayofmonth
    tickets = (ga_tickets
                     #.withColumn('year', year('CREATION_TIME'))
                     #.withColumn('month', month('CREATION_TIME'))
                     #.withColumn('day', dayofmonth('CREATION_TIME'))
                     .where( (concat(col('year'),lpad(col('month'),2, '0'),lpad(col('day'),2, '0'))<=closing_day)
                            &(concat(col('year'),lpad(col('month'),2, '0'),lpad(col('day'),2, '0'))>=starting_day) )
 
                     .join(ga_tickets_detalle, ga_tickets.OBJID==ga_tickets_detalle.OBJID, 'left_outer')
                     .join(ga_franquicia, ga_tickets_detalle.OBJID==ga_franquicia.ID_FRANQUICIA, 'left_outer')
                     #.join(ga_centro_local, ga_tickets.CENTRO_LOCAL==ga_centro_local.ID_CENTRO_LOCAL, 'left_outer')
                     #.join(ga_centro_regional, ga_tickets.CENTRO_REGIONAL==ga_centro_local.ID_CENTRO_REGIONAL, 'left_outer')
                     .join(ga_close_case, ga_tickets.OBJID==ga_close_case.OBJID, 'left_outer')
                     .join(clientes, ga_tickets.CASE_REPORTER2YESTE==clientes.OBJID, 'left_outer')
                     #.join(ga_severidad, ga_tickets.ID_SEVERIDAD==ga_severidad.ID_SEVERIDAD, 'left_outer')
                     .join(ga_tipo_tickets, ga_tickets.ID_TIPO_TICKET==ga_tipo_tickets.ID_TIPO_TICKET, 'left_outer')
                     .filter(col('NIF_CLIENTE').isNotNull())
                     .filter('NIF_CLIENTE != ""')
                     .filter('NIF_CLIENTE != "7"')
          )
 
    averias = tickets.where("X_TIPO_OPERACION IN ('Averia')")
 
    from pyspark.sql.functions import countDistinct
    averias_nif = averias.select('NIF_CLIENTE', 'ID_NUMBER').groupby('NIF_CLIENTE').agg(countDistinct('ID_NUMBER').alias('NUM_AVERIAS_NIF'))
     
    return averias_nif
        

In [ ]:
## Reclamaciones 
def reclamaciones (spark,starting_day,closing_day):
    
    from pyspark.sql.functions import year, month, dayofmonth
    from pyspark.sql.functions import lpad

    interac = (spark.read.table('raw_es.callcentrecalls_interactionono')
                .withColumn('year', year('FX_CREATE_DATE'))
                .withColumn('month', month('FX_CREATE_DATE'))
                .withColumn('day', dayofmonth('FX_CREATE_DATE'))
                .where( (concat(col('year'),lpad(col('month'),2, '0'),lpad(col('day'),2, '0'))<=closing_day)
                       &(concat(col('year'),lpad(col('month'),2, '0'),lpad(col('day'),2, '0'))>=starting_day) )
          )
    clientes = (spark.read.table('raw_es.customerprofilecar_customerow')
                      .select('NUM_CLIENTE', 'NIF_CLIENTE')
                      .withColumnRenamed('NUM_CLIENTE', 'NUM_CLIENTE_cli')
           )
    servicios = spark.read.table('raw_es.customerprofilecar_servicesow').select('NUM_CLIENTE', 'NUM_SERIE')
    serv_cli = servicios.join(clientes, servicios.NUM_CLIENTE==clientes.NUM_CLIENTE_cli, 'full_outer').drop('NUM_CLIENTE_cli')
 
  #  serv_cli.groupBy('NUM_SERIE').agg(count('*')).sort('count(1)', ascending=False).show()
  #  interac.groupBy('DS_X_PHONE_CONSULTATION').agg(count('*')).sort('count(1)', ascending=False).show()

    reclamaciones = (interac.filter(upper(col("DS_REASON_1")).like('%RECLAMA%') | upper(col("DS_RESULT")).like('%RECLAMA%'))
                        .filter(upper(col("DS_DIRECTION")).isin('DE ENTRADA', 'ENTRANTE'))
                        .filter(upper(col("CO_TYPE")).isin('LLAMADA TELEFONO', '...'))
                        .filter(col('DS_X_PHONE_CONSULTATION').isNotNull())
                        .filter('DS_X_PHONE_CONSULTATION != ""')
                        .filter('DS_X_PHONE_CONSULTATION != "000000000"')
                        .join(serv_cli, interac.DS_X_PHONE_CONSULTATION==serv_cli.NUM_SERIE, 'left_outer')
                        .filter(col('NIF_CLIENTE').isNotNull())
                        .filter('NIF_CLIENTE != ""')
                        .filter('NIF_CLIENTE != "7"')
                        #.select('DS_X_PHONE_CONSULTATION', 'CO_INTERACT_ID', 'FX_START_DATE')
                )
    reclamaciones_nif = reclamaciones.select('NIF_CLIENTE', 'CO_INTERACT_ID').groupby('NIF_CLIENTE').agg(countDistinct('CO_INTERACT_ID').alias('NUM_RECLAMACIONES_NIF'))
     
    return reclamaciones_nif
    

In [ ]:
## Soporte Técnico
def soporteTecnico(spark,starting_day,closing_day):
    
    from pyspark.sql.functions import year, month, dayofmonth
    from pyspark.sql.functions import lpad
    from pyspark.sql.functions import broadcast

    interac = (spark.read.table('raw_es.callcentrecalls_interactionono')
                .withColumn('year', year('FX_CREATE_DATE'))
                .withColumn('month', month('FX_CREATE_DATE'))
                .withColumn('day', dayofmonth('FX_CREATE_DATE'))
                .where( (concat(col('year'),lpad(col('month'),2, '0'),lpad(col('day'),2, '0'))<=closing_day)
                       &(concat(col('year'),lpad(col('month'),2, '0'),lpad(col('day'),2, '0'))>=starting_day) )
          )
    clientes = (spark.read.table('raw_es.customerprofilecar_customerow')
                      .select('NUM_CLIENTE', 'NIF_CLIENTE').distinct()
                      .withColumnRenamed('NUM_CLIENTE', 'NUM_CLIENTE_cli')
           )
    servicios = spark.read.table('raw_es.customerprofilecar_servicesow').select('NUM_CLIENTE', 'NUM_SERIE').distinct()
    serv_cli = servicios.join(clientes, servicios.NUM_CLIENTE==clientes.NUM_CLIENTE_cli, 'full_outer').drop('NUM_CLIENTE_cli')
    
    serv_cli=serv_cli.where(serv_cli.NUM_SERIE.isNotNull())
    serv_cli=serv_cli.where(serv_cli.NUM_SERIE != '')
    
    maestro_gt = spark.read.csv('/data/attributes/vf_es/trigger_analysis/maestro_gt/', header=True, sep='\t')
 
    soporte_tmp = (interac.filter(~upper(col("DS_X_GROUP_WORK")).like('%IVR%'))
                      .filter(~upper(col("DS_REASON_2")).like('%Transf%'))
                      .filter(~upper(col("DS_REASON_1")).like('%Transf%'))
                      .join(broadcast(maestro_gt), interac.DS_X_GROUP_WORK==maestro_gt. Grupo_de_Trabajo, 'left_outer')
                      .filter(upper(col("SERVICIO_AGENTE")).isin('CUSTOMER OP.')) 
                  )
    soporte_tmp=soporte_tmp.where(soporte_tmp.DS_X_PHONE_CONSULTATION.isNotNull())
    soporte_tmp=soporte_tmp.where(soporte_tmp.DS_X_PHONE_CONSULTATION != '')
        
    soporte=(soporte_tmp.join(serv_cli, soporte_tmp.DS_X_PHONE_CONSULTATION==serv_cli.NUM_SERIE, 'left_outer')
                      .filter(col('NIF_CLIENTE').isNotNull())
                      .filter('NIF_CLIENTE != ""')
                      .filter('NIF_CLIENTE != "7"')
             )
    
    soporte_nif = soporte.select('NIF_CLIENTE', 'CO_INTERACT_ID').groupby('NIF_CLIENTE').agg(countDistinct('CO_INTERACT_ID').alias('NUM_SOPORTE_TECNICO_NIF'))

    return soporte_nif

    

In [ ]:
def check_partitions (df):
    l = df.rdd.glom().map(len).collect()  # get length of each partition
    print 'Smallest partition {}'.format(min(l))
    print 'Largest partitions {}'.format(max(l))
    print 'Avg. partition size {}'.format(sum(l)/len(l))
    print 'Num partitions {}'.format(len(l))

In [ ]:
def get_customer_base(spark, date_):

    # Customer base at the beginning of the period

    day_ = date_[6:8]
    month_ = date_[4:6]
    year_ = date_[0:4]

    customerDF = spark.read.option("mergeSchema", True)\
    .parquet("/data/udf/vf_es/amdocs_ids/customer/year=" + str(year_) + "/month=" + str(int(month_)) + "/day=" + str(int(day_)))

    serviceDF = spark.read.option("mergeSchema", True)\
    .parquet("/data/udf/vf_es/amdocs_ids/service/year=" + str(year_) + "/month=" + str(int(month_)) + "/day=" + str(int(day_)))\
    
    customerBase = customerDF\
    .join(serviceDF, "NUM_CLIENTE", "inner")\
    .select('NIF_CLIENTE')\
    .dropDuplicates()

    print "[Info Trigger Identification] Number of mobile service in the base of " + date_ + ": " + str(customerBase.count())

    return customerBase


In [ ]:
#starting_day='20190307'
#closing_day='20190307'
#horizon=[2,4,8]

#starting_day_list=['20190430']#['20190314','20190321','20190331','20190407','20190414','20190421','20190430']
closing_day_list=['20190621']#,'20190607']#'20190414','20190421','20190430','20190507','20190514','20190521','20190531']
horizon=[2,4,8]

## Averías

In [ ]:
for closing_day in closing_day_list:
       
    cont=0
    for hh in horizon:
        # Calculamos n ciclos hacia atrás (donde n es el horizon)

        cycle = 0
        fini_tmp = closing_day
        while cycle < hh:
            yearmonthday_target = get_previous_cycle(fini_tmp, str_fmt="%Y%m%d")
            cycle = cycle + 1
            fini_tmp = yearmonthday_target
        starting_day=fini_tmp

        cycle = 0
        fini_tmp = starting_day
        while cycle < hh:
            yearmonthday_target = get_previous_cycle(fini_tmp, str_fmt="%Y%m%d")
            cycle = cycle + 1
            fini_tmp = yearmonthday_target
        ciclo_prev=fini_tmp

        print(ciclo_prev,starting_day,closing_day)

        customerBase=get_customer_base(spark,closing_day)

        dfAverias_ini=averias(spark,starting_day,closing_day)
        dfAverias_ini=dfAverias_ini.withColumnRenamed('NUM_AVERIAS_NIF','NUM_AVERIAS_NIF_ini_w'+str(hh))

        dfAverias_prev=averias(spark,ciclo_prev,starting_day)
        dfAverias_prev=dfAverias_prev.withColumnRenamed('NUM_AVERIAS_NIF','NUM_AVERIAS_NIF_prev_w'+str(hh))

        dfAverias=customerBase.join(dfAverias_ini,on=['NIF_CLIENTE'],how='left_outer')
        dfAverias=dfAverias.join(dfAverias_prev,on=['NIF_CLIENTE'],how='left_outer')

        dfAverias=dfAverias.na.fill(0,subset=['NUM_AVERIAS_NIF_prev_w'+str(hh),'NUM_AVERIAS_NIF_ini_w'+str(hh)])

        dfDifAverias_tmp=dfAverias.withColumn('NUM_AVERIAS_NIF_w'+str(hh)+'vsw'+str(hh),col('NUM_AVERIAS_NIF_ini_w'+str(hh))-col('NUM_AVERIAS_NIF_prev_w'+str(hh)))

        if cont==0: dfDifAverias=dfDifAverias_tmp
        else: dfDifAverias=dfDifAverias.join(dfDifAverias_tmp,on='NIF_CLIENTE',how='left')

        cont=cont+1
        
    dfDifAveriasFin=(dfDifAverias.withColumn('year',lit(int(closing_day[0:4])))
    .withColumn('month',lit(int(closing_day[4:6])))
    .withColumn('day',lit(int(closing_day[6:8]))))
    
    (dfDifAveriasFin.write.partitionBy('year', 'month', 'day').mode("append").format("parquet")
            .save('/data/attributes/vf_es/trigger_analysis/averias/'))
    

## Reclamaciones

In [ ]:
for closing_day in closing_day_list:
    cont=0
    for hh in horizon:
        # Calculamos n ciclos hacia atrás (donde n es el horizon)
        cycle = 0
        fini_tmp = closing_day
        while cycle < hh:
            yearmonthday_target = get_previous_cycle(fini_tmp, str_fmt="%Y%m%d")
            cycle = cycle + 1
            fini_tmp = yearmonthday_target
        starting_day=fini_tmp

        cycle = 0
        fini_tmp = starting_day
        while cycle < hh:
            yearmonthday_target = get_previous_cycle(fini_tmp, str_fmt="%Y%m%d")
            cycle = cycle + 1
            fini_tmp = yearmonthday_target
        ciclo_prev=fini_tmp

        print(ciclo_prev,starting_day,closing_day) 

        customerBase=get_customer_base(spark,closing_day)

        dfReclamaciones_ini=reclamaciones(spark,starting_day,closing_day)
        dfReclamaciones_ini=dfReclamaciones_ini.withColumnRenamed('NUM_RECLAMACIONES_NIF','NUM_RECLAMACIONES_NIF_ini_w'+str(hh))

        dfReclamaciones_prev=reclamaciones(spark,ciclo_prev,starting_day)
        dfReclamaciones_prev=dfReclamaciones_prev.withColumnRenamed('NUM_RECLAMACIONES_NIF','NUM_RECLAMACIONES_NIF_prev_w'+str(hh))

        dfReclamaciones=customerBase.join(dfReclamaciones_ini,on=['NIF_CLIENTE'],how='left_outer')
        dfReclamaciones=dfReclamaciones.join(dfReclamaciones_prev,on=['NIF_CLIENTE'],how='left_outer')

        dfReclamaciones=dfReclamaciones.na.fill(0,subset=['NUM_RECLAMACIONES_NIF_prev_w'+str(hh),'NUM_RECLAMACIONES_NIF_ini_w'+str(hh)])

        dfDifReclamaciones_tmp=dfReclamaciones.withColumn('NUM_RECLAMACIONES_NIF_w'+str(hh)+'vsw'+str(hh),col('NUM_RECLAMACIONES_NIF_ini_w'+str(hh))-col('NUM_RECLAMACIONES_NIF_prev_w'+str(hh)))

        if cont==0: dfDifReclamaciones=dfDifReclamaciones_tmp
        else: dfDifReclamaciones=dfDifReclamaciones.join(dfDifReclamaciones_tmp,on='NIF_CLIENTE',how='left')

        cont=cont+1
        
    dfDifReclamacionesFin=(dfDifReclamaciones.withColumn('year',lit(int(closing_day[0:4])))
    .withColumn('month',lit(int(closing_day[4:6])))
    .withColumn('day',lit(int(closing_day[6:8]))))
    
    (dfDifReclamacionesFin.write.partitionBy('year', 'month', 'day').mode("append").format("parquet")
            .save('/data/attributes/vf_es/trigger_analysis/reclamaciones/'))
    

## Soporte Técnico

In [ ]:
for closing_day in (closing_day_list):

    cont=0
    for hh in horizon:

        # Calculamos n ciclos hacia atrás (donde n es el horizon)
        cycle = 0
        fini_tmp = closing_day
        while cycle < hh:
            yearmonthday_target = get_previous_cycle(fini_tmp, str_fmt="%Y%m%d")
            cycle = cycle + 1
            fini_tmp = yearmonthday_target
        starting_day=fini_tmp

        cycle = 0
        fini_tmp = starting_day
        while cycle < hh:
            yearmonthday_target = get_previous_cycle(fini_tmp, str_fmt="%Y%m%d")
            cycle = cycle + 1
            fini_tmp = yearmonthday_target
        ciclo_prev=fini_tmp

        print(ciclo_prev,starting_day,closing_day)

        customerBase=get_customer_base(spark,closing_day)

        dfSoporteTec_ini=soporteTecnico(spark,starting_day,closing_day)
        dfSoporteTec_ini=dfSoporteTec_ini.withColumnRenamed('NUM_SOPORTE_TECNICO_NIF','NUM_SOPORTE_TECNICO_NIF_ini_w'+str(hh))

        dfSoporteTec_prev=soporteTecnico(spark,ciclo_prev,starting_day)
        dfSoporteTec_prev=dfSoporteTec_prev.withColumnRenamed('NUM_SOPORTE_TECNICO_NIF','NUM_SOPORTE_TECNICO_NIF_prev_w'+str(hh))

        dfSoporteTec=customerBase.join(dfSoporteTec_ini,on=['NIF_CLIENTE'],how='left_outer')
        dfSoporteTec=dfSoporteTec.join(dfSoporteTec_prev,on=['NIF_CLIENTE'],how='left_outer')

        dfSoporteTec=dfSoporteTec.na.fill(0,subset=['NUM_SOPORTE_TECNICO_NIF_prev_w'+str(hh),'NUM_SOPORTE_TECNICO_NIF_ini_w'+str(hh)])

        dfDifSoporteTec_tmp=dfSoporteTec.withColumn('NUM_SOPORTE_TECNICO_NIF_w'+str(hh)+'vsw'+str(hh),col('NUM_SOPORTE_TECNICO_NIF_ini_w'+str(hh))-col('NUM_SOPORTE_TECNICO_NIF_prev_w'+str(hh)))

        if cont==0: dfDifSoporteTec=dfDifSoporteTec_tmp
        else: dfDifSoporteTec=dfDifSoporteTec.join(dfDifSoporteTec_tmp,on='NIF_CLIENTE',how='left')

        cont=cont+1
        
    dfDifSoporteTecFin=(dfDifSoporteTec.withColumn('year',lit(int(closing_day[0:4])))
    .withColumn('month',lit(int(closing_day[4:6])))
    .withColumn('day',lit(int(closing_day[6:8]))))
    
    
    (dfDifSoporteTecFin.write.partitionBy('year', 'month', 'day').mode("append").format("parquet")
            .save('/data/attributes/vf_es/trigger_analysis/soporte_tecnico/'))
        

In [ ]:
dfAv=spark.read.parquet('/data/attributes/vf_es/trigger_analysis/averias/year=2019/month=4/day=14')

In [ ]:
dfAv.filter(col('NUM_AVERIAS_NIF_ini_w8') != col('NUM_AVERIAS_NIF_ini_w2')).count()

In [ ]:
dfAv.count()